In [1]:
import pickle
import seaborn as sns
import networkx as nx
import collections 
import community
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np 

# Source des données 

http://socialcomputing.asu.edu/datasets/Friendster

| Number of Nodes |Number of Edges  |Missing Values? |
|------|------|------|
|100199 |14067887|no|


**Source:**

N/A


**Data Set Information:**

2 files are included:

1. nodes.csv
-- it's the file of all the users. This file works as a dictionary of all the users in this data set. It's useful for fast reference. It contains
all the node ids used in the dataset

2. edges.csv
-- this is the friendship network among the users. The users's friends are represented using edges. Here is an example. 1,2 This means user with id "1" is friend with user id "2".


**Attribute Information:**

Friendster is a social networking website.The service allows users to contact other members, maintain those contacts, and share online content and media with those contacts. 
This is the data set crawled by Stephen Booher (stephen.booher@asu.edu) on Nov, 2010 from Friendster. 
This contains the friendship network crawled. For easier understanding, all the contents are organized in CSV file format.

__Basic statistics__

Number of users : 100,199
Number of friendship pairs: 14,067,887

# Découvertes des données

- Lecture des fichiers téléchargés

In [3]:
#Lecture du fichier des liens
df_edges = pd.read_csv("/home/pb19121/datagraphx/edges_sans_boucle.csv",sep =',',header = None)
df_edges.columns = ['FROM', 'TO']

In [4]:
df_edges.to_csv('/home/pb19121/datagraphx/friendsterallfollowers.txt', sep =  ' ',index = False, header= False)

In [107]:
df_edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14067887 entries, 0 to 14067886
Data columns (total 2 columns):
FROM    int64
TO      int64
dtypes: int64(2)
memory usage: 214.7 MB


In [12]:
df_edges.head(5)

,FROM,TO
0,151838,317823
1,151838,91631
2,151838,91943
3,151838,92060
4,151838,92073


In [6]:
#Lecture du fichier des noeuds
df_nodes = pd.read_csv("/home/pb19121/mydata/nodes.csv",header = None)
df_nodes.columns = ['NODE']

In [9]:
#Lecture du fichier des noeuds
df_nodes_att = pd.read_csv("/home/pb19121/datagraphx/WITH_ATT_NODES.csv",header = None)
df_nodes_att.columns = ['Node','name','Present']

In [11]:
df_nodes_att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100199 entries, 0 to 100198
Data columns (total 3 columns):
Node       100199 non-null int64
name       100199 non-null int64
Present    100199 non-null object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [10]:
df_nodes_att.head(5)

,Node,name,Present
0,87,87,O
1,115,115,O
2,139,139,O
3,270,270,O
4,426,426,O


- Vérification de l'exhaustivité et l'unicité des noeuds

In [13]:
pd.DataFrame(df_nodes.NODE.unique()).count()

0    100199
dtype: int64

In [ ]:
df_TO =pd.DataFrame(df_edges.TO.unique())
df_TO.columns = ['NODE']
df_FROM = pd.DataFrame(df_edges.FROM.unique())
df_FROM.columns = ['NODE']
print("Sommets source :\n")
print(df_FROM.info())
print("Sommets cible : \n") 
print(df_TO.info())
print(df_FROM.head(5) ,"\n", df_TO.head(5))

In [15]:
frames = [df_FROM, df_TO, df_nodes]
df_union_nodes =pd.concat(frames)
df_union_nodes.count()
pd.DataFrame(df_union_nodes.NODE.unique()).count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5689497 entries, 0 to 5689496
Data columns (total 1 columns):
NODE    int64
dtypes: int64(1)
memory usage: 43.4 MB


> **Remarque #1** Tous les noeuds ne figurent pas dans le dictionnaire _"nodes"_

In [33]:
liste = df_nodes.NODE

- Suppression des arretes dont les noeuds sont absents du fichiers nodes.csv

In [ ]:
df_edges_isNOTnode = pd.DataFrame(columns = ['FROM', 'TO'])

In [42]:
df_edges_isNOTnode = df_edges[np.logical_not(df_edges['FROM'].isin(liste) )&(df_edges['TO'].isin(liste))]

In [108]:
df_edges_isNOTnode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397984 entries, 1 to 14067794
Data columns (total 2 columns):
FROM    397984 non-null int64
TO      397984 non-null int64
dtypes: int64(2)
memory usage: 9.1 MB


In [34]:
df_edges_isnode = pd.DataFrame(columns = ['FROM', 'TO'])

In [35]:
df_edges_isnode = df_edges[(df_edges['FROM'].isin(liste) )& (df_edges['TO'].isin(liste))]

In [109]:
df_edges_isnode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981920 entries, 90 to 14067886
Data columns (total 2 columns):
FROM    981920 non-null int64
TO      981920 non-null int64
dtypes: int64(2)
memory usage: 22.5 MB


# Ajout d'attributs calculé sur les noeuds

In [114]:
import pickle
import seaborn as sns
import networkx as nx
import collections 
import community
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy

In [199]:
G_full = nx.Graph()

In [115]:
G_isinDict = nx.Graph()

In [119]:
G_isNOTinDict = nx.Graph()

In [116]:
for  item, row in df_edges_isnode.iterrows():
    G_isinDict.add_edge(row['FROM'],row['TO'], weight=1)    

In [124]:
for  item, row in df_edges_isNOTnode.iterrows():
    G_isNOTinDict.add_edge(row['FROM'],row['TO'], weight=1)    

In [200]:
for  item, row in df_edges.iterrows():
    G_full.add_edge(row['FROM'],row['TO'], weight=1)    

In [118]:
#_SAUVEGARDE DU GRAPHE DANS UN FICHIER
nx.write_edgelist(G_isinDict, "/home/pb19121/mydata/G_isinDict.edgelist")

In [222]:
#_SAUVEGARDE DU GRAPHE DANS UN FICHIER
nx.write_edgelist(G_full, "/home/pb19121/mydata/G_full.edgelist")

In [125]:
#_SAUVEGARDE DU GRAPHE DANS UN FICHIER
nx.write_edgelist(G_isNOTinDict, "/home/pb19121/mydata/G_isNOTinDict.edgelist")

In [ ]:
#_SAUVEGARDE DU GRAPHE DANS UN FICHIER
nx.write_adjlist(G_isinDict, "/home/pb19121/mydata/G_isinDict.adjlist")

In [ ]:
#_SAUVEGARDE DU GRAPHE DANS UN FICHIER
nx.write_adjlist(G_isNOTinDict, "/home/pb19121/mydata/G_isNOTinDict.adjlist")

In [221]:
#_SAUVEGARDE DU GRAPHE DANS UN FICHIER
nx.write_adjlist(G_full, "/home/pb19121/mydata/G_full.adjlist")

In [122]:
print(nx.info(G_isinDict))

Name: 
Type: Graph
Number of nodes: 100199
Number of edges: 981920
Average degree:  19.5994


In [201]:
print(nx.info(G_full))

Name: 
Type: Graph
Number of nodes: 5689498
Number of edges: 14067887
Average degree:   4.9452


In [127]:
print(nx.info(G_isNOTinDict))

Name: 
Type: Graph
Number of nodes: 139584
Number of edges: 1379904
Average degree:  19.7717


# Ajout d'attributs

In [128]:
# Calcul du degres des noeuds (graphe non orienté)
att_deg = G_isinDict.degree()
nx.set_node_attributes(G_isinDict, 'degres', att_deg)

In [129]:
# Calcul du degres des noeuds (graphe non orienté)
att_deg = G_isNOTinDict.degree()
nx.set_node_attributes(G_isNOTinDict, 'degres', att_deg)

In [134]:
att_cluster = nx.clustering(G_isNOTinDict)

In [142]:
type(att_deg)

networkx.classes.reportviews.DegreeView

In [ ]:
nodeliste = G.nodes().to

# Preparation des données Graphx

In [45]:
df_nodes_users = pd.DataFrame(columns = ['USERS'])

In [46]:
df_nodes_users["USERS"] = df_nodes.apply( lambda row :( str(row["NODE"])+",N"+str(row["NODE"])+",n"+str(row["NODE"])) , axis = 1)

In [47]:
df_nodes_users.head(10)

,USERS
0,"1,N1,n1"
1,"2,N2,n2"
2,"3,N3,n3"
3,"4,N4,n4"
4,"5,N5,n5"
5,"7,N7,n7"
6,"11,N11,n11"
7,"12,N12,n12"
8,"13,N13,n13"
9,"14,N14,n14"


In [30]:
df_nodes_users.drop(["NODE"], axis = 1, inplace = True)

In [106]:
df_nodes_users.to_csv('/home/pb19121/datagraphx/friendsterusers.txt', sep =  ' ',index = False, header= False)

In [123]:
df_union_nodes.to_csv('/home/pb19121/datagraphx/friendsterallusers.txt', sep =  ' ',index = False, header= False)

In [108]:
df_edges_isnode.reindex(['TO', 'FROM'], axis=1).to_csv('/home/pb19121/datagraphx/friendsterfollowersReverse.txt', sep =  ' ',index = False, header= False)

In [108]:
df_edges_isnode.to_csv('/home/pb19121/datagraphx/friendsterfollowers.txt', sep =  ' ',index = False, header= False)

In [124]:
df_edges.reindex(['TO', 'FROM'], axis=1).to_csv('/home/pb19121/datagraphx/friendsterallfollowers.txt', sep =  ' ',index = False, header= False)